# Convert FIRE data

This script will convert the FIRE .hdf5 files into a .json data files that can be read into Firefly. 

FIREreader is the class that will allow you to read in files within a directory, create the dictionary, and write out the json files


In [1]:
%load_ext autoreload
%autoreload 2

from firefly_api.FIREreader import FIREreader
import numpy as np

### Set the defaults and create the .json files

In [2]:
reader = FIREreader(
    snapdir = "/Users/agurvich/research/snaps/Control_G4_20/snapdir_050/",
    #snapdir = "/Users/ageller/Visualizations/Firefly/snapdir_050/",
    snapnum = 50,
    ptypes=['PartType0','PartType4'],
    UInames=['Gas','Stars'],
    dec_factors=[10,100],
    returnKeys=['Density','Velocities','Temperature'],
    doMags=[0,0,0],
    doLogs=[1,0,1],
    filterFlags=[1,1,1], ## Note: previously Velocities were automatically included, now you must mark 1 to filter 
    colormapFlags=[1,1,1], 
    JSONdir='isolatedGalaxy_s50')
reader.loadData()

reader.options['color']['Gas']=[1,0,0,1]
reader.options['color']['Stars']=[0,0,1,1]
reader.options['sizeMult']['Gas']=0.3
reader.options['sizeMult']['Stars']=0.3

reader.options['showColormap']['Gas']=True
reader.options['colormapVals']['Gas']['log10Density']=[-2,-1]
reader.options['colormapLims']['Gas']['log10Density']=[-5,1]

reader.dumpToJSON()

FireflyMessage: ('outputting:', 
ParticleGroup: Gas
Contains 1066668 particles (106666 after decimation) and 3 tracked fields)
FireflyMessage: ('outputting:', 
ParticleGroup: Stars
Contains 514118 particles (5141 after decimation) and 1 tracked fields)


## Make an options file to use as a preset

In [3]:
#update a few of the options, here to start by only showing the high-velocity outflows in Gas, as vectors

reader.options['center'] = np.array([-0.11233689678565528, -2.3536859975959175, 0.020126853973307934])
reader.options['camera'] = np.array([12.012246024501222, 16.51869122052115, 1.722756246574182])

reader.options['sizeMult']['Gas'] = 0.4
reader.options['showVel']['Gas'] = True
reader.options['velType']['Gas'] = 'arrow'
reader.options['maxVrange'] = 1000

#Note: if you want to define the filterVals or filterLims above 
#(i.e. to define them before executing reader.run() and after definining reader.addFilter), 
#you would first need to execute reader.defineFilterKeys()  
#(reader.defineFilterKeys() is executed within reader.run() )
reader.options['filterVals']['Gas']['magVelocities'] = [500, 35000]

reader.options['showParts']['Stars'] = False

#This created a file names velocityPreset.json within the data directory 
#  that can now be loaded as a preset from within Firefly

In [4]:
reader.options.listKeys()

-- window_options --
title Firefly
loaded True
-- UI_options --
UI True
UIfullscreen True
UIsnapshot True
UIreset True
UIsavePreset True
UIloadNewData True
UIcameraControls True
UIdecimation True
-- particle_UI_options --
UIparticle {'Gas': True, 'Stars': True}
UIdropdown {'Gas': True, 'Stars': True}
UIcolorPicker {'Gas': True, 'Stars': True}
-- startup_options --
center [-0.1123369  -2.353686    0.02012685]
camera [12.01224602 16.51869122  1.72275625]
cameraRotation None
maxVrange 1000
startFly False
friction None
stereo False
stereoSep None
decimate None
-- particle_startup_options --
plotNmax {'Gas': None, 'Stars': None}
showVel {'Gas': True, 'Stars': False}
velType {'Gas': 'arrow', 'Stars': None}
color {'Gas': [1, 0, 0, 1], 'Stars': [0, 0, 1, 1]}
sizeMult {'Gas': 0.4, 'Stars': 0.3}
showParts {'Gas': True, 'Stars': False}
-- particle_filter_options --
filterVals {'Gas': {'log10Density': None, 'Velocities': None, 'log10Temperature': None, 'magVelocities': [500, 35000]}, 'Stars': {'Ve

In [5]:
reader.options.outputToJSON(reader.JSONdir, "velocityPreset.json")

/Users/agurvich/research/repos/Firefly/data/dataParser.py:238: UserWarning: You will need to add this options filename to filenames.json if this was not called by a Reader instance.
  " filenames.json if this was not called by a Reader instance.")


## For debugging purposes
from firefly_api.options import Options
options = Options()
options.loadFromJSON('preset.json')